In [1]:
import matplotlib.pyplot as plt
import matplotlib

%matplotlib tk
%autosave 180
%load_ext autoreload
%autoreload 2


import numpy as np
#%matplotlib inline

import os
from utils import *

import os

from utils_calcium import ProcessCalcium, get_reward_triggered_psth, plot_multi_session_psth_imshow, plot_psth

Autosaving every 180 seconds


/home/cat/.conda/envs/bmitools/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Importing the dtw module. When using in academic works please cite:
  T. Giorgino. Computing and Visualizing Dynamic Time Warping Alignments in R: The dtw Package.
  J. Stat. Soft., doi:10.18637/jss.v031.i07.



In [22]:
####################################################
################## LOAD DATA #######################
####################################################

root_dir = '/media/cat/8TB/donato/bmi/'

animal_ids = [
    # M1 mice
    "DON-011733",    # M1    - Processed
    "DON-014618",    # M1    - Processed
    'DON-014451',    # M1    - Processed
    'DON-014382',    # M1    - Processed
    'DON-014575',    # M1    - Processed
    'DON-014618',    # M1    - Processed
    'DON-015821',    # M1    - Processed
   # 'DON-017923',   # M1    - Not Processed <has extra days at the end....
    'DON-018129',   # M1    - Not Processed
     'DON-018130',   # M1    - Not Processed

  #  MAYBE MICE
  #  'DON-013392'  <-- only last day was good, all others were at chance
#    'DON-015962'  <-- might be ok, rechedk it.

  #  CA3 mice
    "DON-012502",    # CA3    - spreadsheet is old - need reprocessing; ALSO Weird performance, may exclude
    "DON-014266",    # CA3   - Processed
    'DON-014371',    # CA3   - Processed
    'DON-015801',    # CA3   - Processed
    'DON-016658',    # CA3   - Processed
    "DON-019683",    # CA3   - Processed
]
#

animal_id = animal_ids[0]

#
c = ProcessCalcium(root_dir,
                    animal_id)

# # 
c.remove_bad_cells = False
c.use_non_merged = True

#
c.recompute_binarization = False  # IF TRue: must input correct params from bin notebooks
c.dff_min = 0.05                  # min %DFF for [ca] burst to considered a spike (default 5%) overwrites percentile threshold parameter
c.percentile_threshold = 0.99   # this is pretty fixed, we don't change it; we want [ca] bursts that are well outside the "physics-caused"noise
c.maximum_std_of_signal = 0.25     # if std of signal is greater than this, then we have a noisy signal and we don't want to binarize it

#
c.save_figures = c.recompute_binarization
c.load_sessions()

#
print ("....DONE...")


  0%|          | 0/8 [00:00<?, ?it/s]

 ***** USING non-merged binarization
remove bad cells:  True
...setting 2p parameters...
...loading binarized traces from file:  /media/cat/8TB/donato/bmi/DON-011733/20230302/plane0/binarized_traces.npz


 12%|█▎        | 1/8 [00:01<00:12,  1.75s/it]


 ***** USING non-merged binarization
remove bad cells:  True
...setting 2p parameters...
...loading binarized traces from file:  /media/cat/8TB/donato/bmi/DON-011733/20230303/plane0/binarized_traces.npz


 25%|██▌       | 2/8 [00:03<00:08,  1.50s/it]

...


 ***** USING non-merged binarization
remove bad cells:  True
...setting 2p parameters...
...loading binarized traces from file:  /media/cat/8TB/donato/bmi/DON-011733/20230308/plane0/binarized_traces.npz


 88%|████████▊ | 7/8 [00:32<00:03,  3.97s/it]


 ***** USING non-merged binarization
remove bad cells:  True
...setting 2p parameters...
...loading binarized traces from file:  /media/cat/8TB/donato/bmi/DON-011733/20230309/plane0/binarized_traces.npz


100%|██████████| 8/8 [00:34<00:00,  4.25s/it]


....DONE...


In [24]:
##############################################################
######## COMPUTE REWARD TRIGGERED RESPONSE OF CELLS ##########
##############################################################

#
for animal_id in animal_ids[1:]:

    #
    c = ProcessCalcium(root_dir,
                        animal_id)
    
    # 
    fname_out = os.path.join(c.root_dir,
                            c.animal_id,
                            'results',
                            c.animal_id+'_'+c.rec_type+'_multi_session_psth_imshow.svg')

    if os.path.exists(fname_out):
        print ("File exists: ", fname_out)
        continue

    # # 
    c.remove_bad_cells = False
    c.use_non_merged = True

    #
    c.recompute_binarization = False  # IF TRue: must input correct params from bin notebooks
    c.dff_min = 0.05                  # min %DFF for [ca] burst to considered a spike (default 5%) overwrites percentile threshold parameter
    c.percentile_threshold = 0.99   # this is pretty fixed, we don't change it; we want [ca] bursts that are well outside the "physics-caused"noise
    c.maximum_std_of_signal = 0.25     # if std of signal is greater than this, then we have a noisy signal and we don't want to binarize it

    #
    c.save_figures = c.recompute_binarization
    c.load_sessions()

    window = 30*6
    psth_array = []
    idx_cells = None
    global_order = False
    session_ids = np.arange(1,len(c.session_ids),1)

    #
    n_bursts_array = []

    global_order = False
    for session_id in session_ids:
    #for session_id in [1,7]:


        #
        psths_avg, psths_shuffled_avg, idx_cells, n_bursts  = get_reward_triggered_psth(session_id, 
                                                                                        c,
                                                                                        window,
                                                                                        idx_cells,
                                                                                        global_order)
    
        #
        n_bursts_array.append(n_bursts)
        
        #
        try:
            psth_array.append(psths_avg[idx_cells[::-1]])
        except:
            print ("Error appending psth array")
            continue
        if global_order==False:
            idx_cells = None

    #
    #psth_array = np.array(psth_array)
    #print (psth_array.shape)

    #

    vmax = 0.3

    sua = plot_multi_session_psth_imshow(c,
                                session_ids,
                                psth_array,
                                window,
                                n_bursts_array,
                                vmax=vmax)


File exists:  /media/cat/8TB/donato/bmi/DON-014618/results/DON-014618_m1_multi_session_psth_imshow.svg
File exists:  /media/cat/8TB/donato/bmi/DON-014451/results/DON-014451_m1_multi_session_psth_imshow.svg
File exists:  /media/cat/8TB/donato/bmi/DON-014382/results/DON-014382_m1_multi_session_psth_imshow.svg
File exists:  /media/cat/8TB/donato/bmi/DON-014575/results/DON-014575_m1_multi_session_psth_imshow.svg
File exists:  /media/cat/8TB/donato/bmi/DON-014618/results/DON-014618_m1_multi_session_psth_imshow.svg


  0%|          | 0/9 [00:00<?, ?it/s]

 ***** USING non-merged binarization
remove bad cells:  True
...setting 2p parameters...
...loading binarized traces from file:  /media/cat/8TB/donato/bmi/DON-015821/20230519/plane0/binarized_traces.npz


 11%|█         | 1/9 [00:01<00:12,  1.60s/it]


 ***** USING non-merged binarization
remove bad cells:  True
...setting 2p parameters...
...loading binarized traces from file:  /media/cat/8TB/donato/bmi/DON-015821/20230520/plane0/binarized_traces.npz


...

 89%|████████▉ | 8/9 [00:10<00:01,  1.15s/it]


 ***** USING non-merged binarization
remove bad cells:  True
...setting 2p parameters...
...loading binarized traces from file:  /media/cat/8TB/donato/bmi/DON-015821/20230527/plane0/binarized_traces.npz


100%|██████████| 9/9 [00:13<00:00,  1.46s/it]


/home/cat/code/bmitools/bmitools/analysis/calcium/networks/utils_calcium.py:5195: RuntimeWarning: Mean of empty slice
  psths_avg = np.nanmean(psths,1)
/home/cat/code/bmitools/bmitools/analysis/calcium/networks/utils_calcium.py:5196: RuntimeWarning: Mean of empty slice
  psths_shuffled_avg = np.nanmean(psths_shuffled,1)


TypeError: 'NoneType' object is not subscriptable